In [1]:
#!pip install matplotlib
#!pip install .

In [2]:
from comorbid_graphs import ComorbidGraph, ComorbidGraphNode
from anytree import PostOrderIter

### Creating graph 

In [3]:
import json
with open('../tests/fixtures/symp_tree.json') as f:
    data = json.load(f)
simple_cg = ComorbidGraph(data, ComorbidGraphNode, assign_ids=True)

In [4]:
simple_cg.tree

ComorbidGraphNode('/Source', id=1, score=0, type='default')

In [5]:
result_cg = simple_cg.simple_search("nervous,headache", ComorbidGraphNode)
# result_cg.tree.children

In [6]:
from anytree import RenderTree

for row in RenderTree(result_cg.tree, childiter=result_cg.order_by_score):
    print("%s%s" % (row.pre, row.node.name))

search_results
├── hallucination
├── abnormality of gait
├── ataxia
├── dysarthria
└── nervous system symptom
    ├── stroke
    ├── paralysis
    ├── meningismus
    ├── aphasia
    ├── meningitis
    ├── meningoencephalitis
    ├── spondylitis
    ├── behavioral symptom
    └── sensation perception
        ├── nociceptive pain
        └── headache
            ├── bifrontal headache
            ├── frontal headache
            └── severe headache


## Searching
Searching should be extensible to allow sql search for larger data than KGs.
For this we have a many step search.
- search for ancestors, parents first
  - get the names of these, include-exclude
  - merge all of the nodes, merging with include-exclude
- for docs in the ending KnowledgeGraph
  - filter by type
  - filter by body length
- search in the body

In [7]:
with open('../tests/fixtures/symp_tree.json') as f:
    data = json.load(f)
cg = ComorbidGraph(data, ComorbidGraphNode, assign_ids=True, root_name='symp_tree')
test_node = cg.tree.children[0]
test_node_content = [i for i in PostOrderIter(cg.tree) if hasattr(i, 'body')][0]

### 1. Filtering
Filter based on node properties if included or not - `name`, `parent`, `type`, `body-length`, `content`.


In [8]:
assert test_node._score_name([test_node.name.lower()], [])
assert test_node._score_parent([test_node.parent.name.lower()],[])
assert test_node._score_type(['default'],[])

# text-length checks
# check first node that has no content
assert not cg.tree._score_text_length([0],[])

# check a node with content
assert test_node_content._score_text_length([0],[])
assert test_node_content._score_content([test_node_content.body.lower()],[])


### 2. Content
Filtering the body for keywords.
But it shouldnt be like the previous case, because loading the body of many of these guys will tire the machine - so will have to allow a filtering done by an en engine (like `sqlite`-engine) which are optimized for these kinds of updates.

In [9]:
assert test_node_content.apply_lbl_content_filter(
    {"content": {"inc":[test_node_content.body.lower()], "exc":[]}}
)

### 3. Subgraph
Filter the `ancestors`, and use `inc-exc` to zoom in.

In [10]:
subgraph = cg.filter_subgraph(
    inc_list=['nervous system'], exc_list=['pain', 'aphasia', 'sensation perception'], base_name='subgraph results'
)
for pre, fill, node in RenderTree(subgraph):
    if hasattr(node, 'old_parent') and hasattr(node.old_parent, 'name'):
        print("%s%s - %s" % (pre, node.name, node.old_parent.name))
    else:
        print("%s%s" % (pre, node.name))

subgraph results
└── nervous system symptom - subgraph results
    ├── coordination symptom
    │   ├── lack of coordination
    │   │   └── incoordination
    │   └── impaired coordination
    ├── paralysis
    │   ├── transient paralysis of limb
    │   ├── motor paralysis
    │   ├── spastic paralysis
    │   ├── paraplegia
    │   ├── lip paralysis
    │   ├── throat muscle paralysis
    │   ├── ophthalmoplegia
    │   ├── proximal paralysis of arm and leg
    │   ├── respiratory paralysis
    │   ├── hind limb paralysis
    │   │   └── partial hind limb paralysis
    │   ├── pareses
    │   ├── extraocular muscles paralysis
    │   ├── facial paralysis
    │   ├── flaccid paralysis
    │   └── hemiparesis
    ├── behavioral symptom
    │   ├── hyperactivity
    │   ├── limited attention
    │   ├── communication difficulty
    │   ├── obsessive interests
    │   ├── repetitive behavior
    │   ├── irritability
    │   ├── abnormal behavior
    │   └── aggressive behavior
    ├── m

In [11]:
nodes_list = set([i for i in PostOrderIter(subgraph)])-set([subgraph])
result_node = cg.merge_nodes_into_tree(nodes_list, node_type=ComorbidGraphNode)

for pre, fill, node in RenderTree(result_node):
    if hasattr(node, 'old_parent') and hasattr(node.old_parent, 'name'):
        print("%s%s - %s" % (pre, node.name, node.old_parent.name))
    else:
        print("%s%s" % (pre, node.name))

source
└── nervous system symptom - subgraph results
    ├── coordination symptom
    │   ├── lack of coordination
    │   │   └── incoordination
    │   └── impaired coordination
    ├── paralysis
    │   ├── transient paralysis of limb
    │   ├── motor paralysis
    │   ├── spastic paralysis
    │   ├── paraplegia
    │   ├── lip paralysis
    │   ├── throat muscle paralysis
    │   ├── ophthalmoplegia
    │   ├── proximal paralysis of arm and leg
    │   ├── respiratory paralysis
    │   ├── hind limb paralysis
    │   │   └── partial hind limb paralysis
    │   ├── pareses
    │   ├── extraocular muscles paralysis
    │   ├── facial paralysis
    │   ├── flaccid paralysis
    │   └── hemiparesis
    ├── behavioral symptom
    │   ├── hyperactivity
    │   ├── limited attention
    │   ├── communication difficulty
    │   ├── obsessive interests
    │   ├── repetitive behavior
    │   ├── irritability
    │   ├── abnormal behavior
    │   └── aggressive behavior
    ├── meningismus

## Searchable
Create the search language by allowing all entries.   
Control for inputs irregularities and more.

In [12]:
query_str = """nervous system"""
print(cg.build_query(query_str))
result_cg = cg.advanced_search(query_str, node_type=ComorbidGraphNode, with_children=True)

print(result_cg.explore())

{'content': {'inc': ['nervous system'], 'exc': []}}
search results
├── hallucination
├── dysarthria
├── abnormality of gait
│   └── wobble
├── sensation perception
│   ├── pain
│   │   ├── abdominal pain
│   │   │   ├── periumbilic abdominal pain
│   │   │   ├── left lower quadrant abdominal pain
│   │   │   ├── right upper quadrant abdominal pain
│   │   │   ├── right lower quadrant abdominal pain
│   │   │   ├── epigastric abdominal pain
│   │   │   ├── multiple sites abdominal pain
│   │   │   ├── generalized abdominal pain
│   │   │   └── left upper quadrant abdominal pain
│   │   ├── backache
│   │   │   ├── low backache
│   │   │   ├── inflammatory low back pain
│   │   │   └── severe backache
│   │   ├── joint pain
│   │   │   ├── hip pain
│   │   │   ├── severe joint pain
│   │   │   ├── knee pain
│   │   │   ├── shoulder pain
│   │   │   └── elbow pain
│   │   ├── headache
│   │   │   ├── frontal headache
│   │   │   ├── severe headache
│   │   │   └── bifrontal headache
│   │

## Ordering Results
There should be two options - first the graph properties.  
Second our simple algorithm based on combination of scores - as found in `comorbid-lab`.

In [13]:
with open('../tests/fixtures/symp_tree.json') as f:
    data = json.load(f)
cg = ComorbidGraph(data, ComorbidGraphNode, assign_ids=True)

In [18]:
query_str = """
pain
inc_ancestor:nervous system symptom
"""
query_dict = cg.build_query(query_str)

In [19]:

result_cg = cg.advanced_search(query_str, base_name="search", node_type=ComorbidGraphNode, with_children=True)
print(result_cg.explore())

search
├── backache
├── headache
├── muscle pain
├── pleuritic chest pain
├── throat pain
├── claudication
├── heartburn
├── colic
│   └── renal colic
├── neck pain
├── chronic pain
├── breakthrough pain
├── visceral pain
├── nociceptive pain
├── neuropathic pain
├── acute pain
├── phantom pain
├── allodynia
├── referred pain
├── colicky pain
├── hypoalgesia
└── hyperalgesia



In [20]:
query_str = """
inc_name:symptom
inc_ancestor:nervous system symptom
"""
result_symptom_cg = cg.advanced_search(query_str, node_type=ComorbidGraphNode, with_children=True)
print(result_symptom_cg.explore())

search results
└── nervous system symptom
    ├── coordination symptom
    ├── behavioral symptom
    └── reflex symptom

